In [73]:
import pandas as pd
import pprint as pp
import numpy as np

In [74]:
code =  {'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 
         'Arkansas': 'AR', 'California': 'CA', 
         'Colorado': 'CO', 'Connecticut': 'CT',
         'Delaware': 'DE', 'District of Columbia': 'DC', 'Florida': 'FL', 'Georgia': 'GA', 
         'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 
         'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY' , 'Louisiana': 'LA',
         'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA',
         'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 
         'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 
         'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 
         'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 
         'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 
         'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 
         'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 
         'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}

In [75]:
df = pd.read_csv('../data/AllGunShootings.csv') 

# calculate the amount of casualties for each incident and insert into dataframe
df.insert(value = [(kill + inj) for kill, inj in zip(df['n_killed'], 
          df['n_injured'])], loc= df.shape[1], column = 'casualties')

# calculate the sum of all casualties per state
df['total_casualties_state'] = df.groupby(['state'])['casualties'].transform(sum)
df['total_injured_state'] = df.groupby(['state'])['n_injured'].transform(sum)
df['total_killed_state'] = df.groupby(['state'])['n_killed'].transform(sum)

# calculate amount of casualties per 10000 citizens and insert into dataframe
df.insert(value = [(total_casualties_state / state_population * 10000) for total_casualties_state, state_population in zip(df['total_casualties_state'],
          df['state_population'])], loc= df.shape[1], column = 'casualties_per_10000_citizens')

# calculate percentage of killed casualties and insert into dataframe
df.insert(value = [(total_killed_state / total_casualties_state * 100) for total_killed_state, total_casualties_state in zip(df['total_killed_state'],
          df['total_casualties_state'])], loc= df.shape[1], column = 'killed_percentage_state')

# calculate percentage of injured casualties and insert into dataframe
df.insert(value = [(total_injured_state / total_casualties_state * 100) for total_injured_state, total_casualties_state in zip(df['total_injured_state'],
          df['total_casualties_state'])], loc= df.shape[1], column = 'injured_percentage_state')

# group the data together for each corresponding state
df = pd.DataFrame({'count' : df.groupby( [ 'state', 'casualties_per_10000_citizens', 'killed_percentage_state', 'injured_percentage_state', 'total_casualties_state', 'total_injured_state', 'total_killed_state', 'state_population', 'state_gun_laws'] ).size()}).reset_index()

# insert the code for each state, so it is supported by the map plot
df.insert(value = [code[df.iloc[i,0]] for i in range(df.shape[0])], loc = 0, column = 'code')

# delete useless column
del df['count']

# delete the row with District of Colombia, because this state is not supported by the map plot
df = df[~df['state'].isin(['District of Columbia'])]

# round the population count to millions
df['state_population'] = df.apply(lambda row: str(round(row['state_population'] / 1000000,2)) + 'M', axis=1)

# round all floats to floats with two decimals
df = df.round(2)

# export dataframe to csv
file_name= '../data/casualties_per_10000_citizens.csv'
df.to_csv(file_name)